# LCO Static 2-Tier Demo (Pyomo)

This notebook implements the **static** two-tier Lexicographic Constraint Optimization (LCO) demo:

- **Tier \(\mathcal{L}_2\)**: Maximize expected revenue  
- **Tier \(\mathcal{L}_3\)**: Minimize expected overbooking slack \(\sum_d w_d\)  
  subject to a **revenue floor** \( \mathcal{L}_2(\mathbf{x}) \ge Z_2^* - \varepsilon_2 \).

The model is the same 10 rooms × 5 days, 12 bookings example used in the main paper.


In [ ]:
# If running in Colab, uncomment the following:
# !pip install pyomo highspy
# !apt-get install -y coinor-cbc

from pyomo.environ import (
    ConcreteModel, Set, Var, Param, Binary, NonNegativeReals,
    Constraint, Objective, maximize, minimize, value, SolverFactory
)


In [ ]:
# -----------------------------
# Synthetic data
# -----------------------------

DAYS  = 5
ROOMS = 10
days  = list(range(1, DAYS+1))
rooms = list(range(1, ROOMS+1))

# 12 bookings: (start_day, length_of_stay, price_per_night, show_probability)
bookings = {
    1:  (1, 2, 120, 0.92),
    2:  (1, 3, 110, 0.85),
    3:  (2, 2, 150, 0.90),
    4:  (2, 3, 130, 0.80),
    5:  (3, 2, 140, 0.88),
    6:  (3, 3, 100, 0.83),
    7:  (4, 2, 160, 0.87),
    8:  (4, 2, 115, 0.78),
    9:  (5, 1, 200, 0.95),
    10: (1, 1, 180, 0.90),
    11: (2, 1, 170, 0.82),
    12: (3, 1, 175, 0.89),
}

def stay_days(bid):
    s, L, _, _ = bookings[bid]
    return list(range(s, min(s+L, DAYS+1)))

B = list(bookings.keys())

# Daily capacity: here, constant = ROOMS for all days
CAP = {d: ROOMS for d in days}

# -----------------------------
# Model builder
# -----------------------------

def build_static_lco_model():
    m = ConcreteModel()

    # Sets
    m.B = Set(initialize=B)
    m.R = Set(initialize=rooms)
    m.D = Set(initialize=days)

    # Parameters
    m.start = Param(m.B, initialize={b: bookings[b][0] for b in B})
    m.len   = Param(m.B, initialize={b: bookings[b][1] for b in B})
    m.price = Param(m.B, initialize={b: bookings[b][2] for b in B})
    m.showp = Param(m.B, initialize={b: bookings[b][3] for b in B})
    m.cap   = Param(m.D, initialize=CAP)

    # Helper sets
    in_stay = {(b, d) for b in B for d in days if d in stay_days(b)}
    m.InStay = Set(dimen=2, initialize=in_stay)

    yidx = {(b, r, d) for (b, d) in in_stay for r in rooms}
    m.YIDX = Set(dimen=3, initialize=yidx)

    cont = {(b, r, d) for b in B for r in rooms
            for d in days if d in stay_days(b) and (d+1) in stay_days(b)}
    m.ContPair = Set(dimen=3, initialize=cont)

    # Variables
    m.a = Var(m.B, within=Binary)                # accept booking
    m.y = Var(m.YIDX, within=Binary)             # assignment
    m.w = Var(m.D,   within=NonNegativeReals)    # overbooking slack

    # Constraints

    # 1) Room exclusivity
    def room_excl(m, r, d):
        return sum(m.y[b, r, d] for b in m.B if (b, d) in m.InStay) <= 1
    m.RoomExcl = Constraint(m.R, m.D, rule=room_excl)

    # 2) Acceptance/assignment link
    def assign_if_accepted(m, b, d):
        if (b, d) not in m.InStay:
            return Constraint.Skip
        return sum(m.y[b, r, d] for r in m.R) == m.a[b]
    m.Assign = Constraint(m.B, m.D, rule=assign_if_accepted)

    # 3) Continuity across stay
    def continuity(m, b, r, d):
        return m.y[b, r, d] == m.y[b, r, d+1]
    m.Continuity = Constraint(m.ContPair, rule=continuity)

    # 4) Overbooking slack: w_d >= expected_shows_d - cap_d
    def overbooking_slack(m, d):
        expected = sum(m.a[b] * m.showp[b] for b in m.B if (b, d) in m.InStay)
        return m.w[d] >= expected - m.cap[d]
    m.OverbookingSlack = Constraint(m.D, rule=overbooking_slack)

    # Revenue expression (Tier L2)
    m.RevenueExpr = sum(m.a[b] * m.price[b] * m.len[b] for b in m.B)

    # Initial objective: maximize revenue (Tier L2)
    m.obj = Objective(expr=m.RevenueExpr, sense=maximize)

    return m


In [ ]:
# -----------------------------
# Solve Tier L2 (maximize revenue)
# -----------------------------

m = build_static_lco_model()

# Choose a solver
solver_name = "highs"  # or "cbc", "gurobi", etc. if installed
opt = SolverFactory(solver_name)

print("Solving Tier L2 (revenue maximization)...")
res_L2 = opt.solve(m, tee=False)
Z2 = value(m.RevenueExpr)
print(f"Tier L2 optimum revenue Z2* = {Z2:.2f}")

# -----------------------------
# Inject revenue floor & switch to Tier L3
# -----------------------------

eps = 1e-6
m.RevenueFloor = Constraint(expr=m.RevenueExpr >= Z2 - eps)

# Replace objective with sum of overbooking slack
m.del_component(m.obj)
m.obj = Objective(expr=sum(m.w[d] for d in m.D), sense=minimize)

print("Solving Tier L3 (overbooking minimization with revenue floor)...")
res_L3 = opt.solve(m, tee=False)
Z3 = sum(value(m.w[d]) for d in m.D)
print(f"Tier L3 optimum total overbooking slack = {Z3:.4f}")


In [ ]:
# -----------------------------
# Summarize accepted bookings and room assignments
# -----------------------------

assignments = []
for b in m.B:
    if value(m.a[b]) > 0.5:
        sdays = stay_days(b)
        assigned_r = None
        for r in m.R:
            ok = True
            for d in sdays:
                if (b, r, d) not in m.YIDX or value(m.y[b, r, d]) <= 0.5:
                    ok = False
                    break
            if ok:
                assigned_r = r
                break
        assignments.append((b, sdays, assigned_r))

print("Accepted bookings and their assigned room (constant across stay):")
for b, sdays, r in assignments:
    print(f"  Booking {b:2d} -> Room {r}, Days {sdays}")

print("\nDaily overbooking slack w_d:")
for d in m.D:
    print(f"  Day {d}: w_d = {value(m.w[d]):.4f}")
